<a href="https://colab.research.google.com/github/surendrareddypy/MLProjects/blob/main/complexword_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
from functools import lru_cache
import re
import unicodedata
import sys
from collections import Counter
import nltk
nltk.download('brown')
from nltk.corpus import brown
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
pip install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1322 sha256=f29410755e09ca126a2e822385e705de8ccc9dcb1477080bf074af05b2c35e3e
  Stored in directory: /root/.cache/pip/wheels/90/bd/0f/a2797a7e90de1cdd91548fba3b16a7c2560e14a268bb137101
Successfully built unzip


In [6]:
Dataset = namedtuple('Dataset', 'name, train, test')
Model = namedtuple('Model', 'type, name, dimension, corpus, model')

In [21]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
MAIN_PATH_DATASET = "traindevset/english/"
genres = ['Wikipedia', 'WikiNews', 'News']
datasets = ['Train', 'Dev']
columns = ['id', 'sentence', "start", "end", "target", 
           "nat", "non_nat", "nat_marked", "non_nat_marked", "binary", "prob"]


datasets = [Dataset('Wikipedia', 'Train', 'Dev'),
            Dataset('WikiNews', 'Train', 'Dev'),
            Dataset('News', 'Train', 'Dev')]

feature_categories = []

def load_df(path):
    df = pd.read_csv(path, header=None, sep = "\t")
    df.columns = columns
    return df

datasets = [Dataset(d.name, load_df(MAIN_PATH_DATASET + d.name + '_' + d.train + '.tsv'),
                            load_df(MAIN_PATH_DATASET + d.name + '_' + d.test + '.tsv'))
                            for d in datasets]

In [22]:
datasets


[Dataset(name='Wikipedia', train='Train', test='Dev'),
 Dataset(name='WikiNews', train='Train', test='Dev'),
 Dataset(name='News', train='Train', test='Dev')]

In [26]:
dataframe = datasets[0].train[0:30]

def overlaps(start1, end1, start2, end2):
    return bool(range(max(start1, start2), min(end1, end2)+1))

def extract_ngrams_group(group):
    targets = zip(group['target'].values.tolist(), group['start'].values.tolist(),
                 group['end'].values.tolist(), group['binary'].values.tolist())
    for word, start, end, binary in targets:
        tokens = word.split()
        if len(tokens)>1:
            olap_words = [(w, b) for w, s, e, b in targets if overlaps(start, end, s, e)]
            
    
grouped = dataframe.groupby('sentence').apply(lambda group : extract_ngrams_group(group))

In [27]:
wordlist_lowercased = set(i.lower() for i in brown.words())
print (len(wordlist_lowercased))

49815


In [59]:
datasets.append(Dataset('train_all_test_wiki', datasets[0].train.append(datasets[1].train).append(datasets[2].train), datasets[0].test))

AttributeError: ignored

In [51]:
# Append train and test set
dataset_sel = datasets[3]
train_num_rows = dataset_sel.train.shape[0]
train_num_sents = len(list(set(dataset_sel.train.sentence.values.tolist())))

test_num_rows = dataset_sel.test.shape[0]
test_num_sents = len(list(set(dataset_sel.test.sentence.values.tolist())))

dataset = dataset_sel.train.append(dataset_sel.test)
dataset['sent_id'] = dataset.groupby('sentence').ngroup()
dataset_num_rows = dataset.shape[0]
dataset_num_sents = len(list(set(dataset.sentence.values.tolist())))

print('# Rows train : {}'.format(train_num_rows))
print('# Rows test : {}'.format(test_num_rows))
print('# Rows dataset : {}'.format(dataset_num_rows))

print('# Sents train : {}'.format(train_num_sents))
print('# Sents test : {}'.format(test_num_sents))
print('# Sents dataset : {}'.format(dataset_num_sents))

IndexError: ignored

In [ ]:
dataset

,id,sentence,start,end,target,nat,non_nat,nat_marked,non_nat_marked,binary,prob,sent_id
0,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",0,8,Normally,10,10,0,1,1,0.05,1347
1,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",28,34,passed,10,10,0,1,1,0.05,1347
2,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",15,19,land,10,10,0,0,0,0.00,1347
3,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,49,future,10,10,1,0,1,0.05,1347
4,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,61,future generations,10,10,1,2,1,0.15,1347
...,...,...,...,...,...,...,...,...,...,...,...,...
689,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,100,109,knowledge,10,10,0,2,1,0.10,929
690,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,74,80,person,10,10,0,0,0,0.00,929
691,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,95,99,true,10,10,0,0,0,0.00,929
692,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,113,122,purifying,10,10,7,8,1,0.75,929


In [ ]:
train, test = train_test_split(dataset[0:10], test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import nltk
nltk.download('stopwords')
# Load pre-trained word embeddings
embeddings_path = '/content/glove.6B.100d.txt'
word_vectors = KeyedVectors.load_word2vec_format(embeddings_path, binary=False)

stop_words = set(stopwords.words('english'))
def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]

# Convert text columns to numerical vectors using pre-trained word embeddings
train_vectors = []
for sentence, target in zip(train['sentence'], train['target']):
    sentence_vector = np.mean([word_vectors[w] for w in sentence.split() if preprocess(w) in word_vectors.vocab], axis=0)
    target_vector = np.mean([word_vectors[w] for w in target.split() if w in word_vectors.vocab], axis=0)
    train_vectors.append(np.concatenate((sentence_vector, target_vector)))

test_vectors = []
for sentence, target in zip(test['sentence'], test['target']):
    sentence_vector = np.mean([word_vectors[w] for w in sentence.split() if w in word_vectors.vocab], axis=0)
    target_vector = np.mean([word_vectors[w] for w in target.split() if w in word_vectors.vocab], axis=0)
    test_vectors.append(np.concatenate((sentence_vector, target_vector)))

# Normalize the data
scaler = StandardScaler()
train_vectors = scaler.fit_transform(train_vectors)
test_vectors = scaler.transform(test_vectors)

# Train the SVM model
X_train = [train_vectors,train["start"],train["end"] , train["nat"],train["non_nat"] , train["nat_marked"], train["non_nat_marked"]]
y_train = train['binary']
svm_model = svm.SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train, y_train)

# Make predictions on test data
X_test = [test_vectors,test["start"],test["end"] , test["nat"],test["non_nat"] , test["nat_marked"], test["non_nat_marked"]]
y_test = test['binary']
y_pred = svm_model.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: ignored

In [ ]:

from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Train the Word2Vec model on the training data
sentences = train['sentence'].apply(lambda x: x.split()).tolist()
w2v_model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)

# Convert each sentence into a vector by averaging the vectors of its words
train_vec = train['sentence'].apply(lambda x: pd.Series(w2v_model.wv[x.split()].mean(axis=0)))
test_vec = test['sentence'].apply(lambda x: pd.Series(w2v_model.wv[x.split()].mean(axis=0)))

# Add the target feature to the numerical features
train_features = pd.concat([train_vec, train.iloc[:, 2:]], axis=1)
test_features = pd.concat([test_vec, test.iloc[:, 2:]], axis=1)

# Train the SVM model
clf = SVC(kernel='linear')
clf.fit(train_features, train['complex_word'])

# Make predictions on the test set
predictions = clf.predict(test_features)

# Evaluate the model performance
accuracy = accuracy_score(test['complex_word'], predictions)
print('Accuracy:', accuracy)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)


In [ ]:

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV,StratifiedKFold

In [ ]:
import pandas as pd
import numpy as np
import gensim

# Load pre-trained Word2Vec model
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('path/to/word2vec/model', binary=True)

# Define function to convert a sentence to a vector using Word2Vec model
def sentence_to_vec(sentence, model):
    """
    Convert a sentence to a vector using a pre-trained Word2Vec model.
    """
    words = sentence.split()
    vecs = []
    for word in words:
        try:
            vecs.append(model[word])
        except KeyError:
            # Ignore out-of-vocabulary words
            pass
    return np.mean(vecs, axis=0)

# Load dataset with text and numerical columns
data = pd.read_csv('path/to/dataset.csv')

# Convert text columns to numerical vectors using Word2Vec model
text_cols = ['sentence', 'target']
for col in text_cols:
    data[col+'_vec'] = data[col].apply(lambda x: sentence_to_vec(x, w2v_model))

# Combine numerical features with text vectors
numerical_cols = ['feature1', 'feature2']
X = np.concatenate([data[col].values.reshape(-1, 1) for col in numerical_cols] + 
                    [np.array(data[col].values.tolist()) for col in text_cols+'_vec'], axis=1)

# Define labels
y = data['label'].values

# Train SVM model
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X, y)

In [ ]:

# Create Function Transformer to use Feature Union
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)
numerc_features=[dataset["start"],dataset["end"] , dataset["nat"],dataset["non_nat"] , dataset["nat_marked"], dataset["non_nat_marked"],dataset["binary"]]
text_features=[dataset['sentence'],dataset['target']]
# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

# Training config
kfold = StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(train_data, train_labels)
rf_best = rf_model.best_estimator_

Fitting 7 folds for each of 20 candidates, totalling 140 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(


ValueError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# fit the vectorizer on the sentence and target columns
vectorizer.fit(dataset['sentence'].append(dataset['target']))

# transform the sentence and target columns to TF-IDF vectors
train_sentence_tfidf = vectorizer.transform(dataset['sentence'])
train_target_tfidf = vectorizer.transform(dataset['target'])

In [ ]:
train_sentence_tfidf

<27993x7669 sparse matrix of type '<class 'numpy.float64'>'
	with 677310 stored elements in Compressed Sparse Row format>

In [ ]:
# Split the dataset into training, validation, and test sets
train_data, test_data, train_labels, test_labels = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Preprocess the data
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

# Train the SVM model
svm = SVC(C=1.0, kernel='rbf', gamma='scale')
svm.fit(train_data, train_labels)

# Validate the model
scores = cross_val_score(svm, val_data, val_labels, cv=5, scoring='accuracy')
print('Cross-validation scores:', scores)
print('Mean cross-validation score:', scores.mean())

# Test the model
pred_labels = svm.predict(test_data)
print('Test accuracy:', accuracy_score(test_labels, pred_labels))
print('Test precision:', precision_score(test_labels, pred_labels))
print('Test recall:', recall_score(test_labels, pred_labels))
print('Test F1 score:', f1_score(test_labels, pred_labels))

ValueError: ignored

In [ ]:
# Load each CSV file into a separate Pandas DataFrame
news_train = pd.read_csv('/content/traindevset/english/News_Train.tsv',sep='\t')
news_dev = pd.read_csv('/content/traindevset/english/News_Dev.tsv',sep='\t')
wikinews_dev = pd.read_csv('/content/traindevset/english/WikiNews_Dev.tsv',sep='\t')
wikinews_train = pd.read_csv('/content/traindevset/english/WikiNews_Train.tsv',sep='\t')
wikipedia_dev= pd.read_csv('/content/traindevset/english/Wikipedia_Dev.tsv',sep='\t')
wikipedia_train= pd.read_csv('/content/traindevset/english/Wikipedia_Train.tsv',sep='\t')
news_test=pd.read_csv('/content/testset/english/News_Test.tsv',sep='\t')
wikinews_test=pd.read_csv('/content/testset/english/WikiNews_Test.tsv',sep='\t')
wikipedia_test=pd.read_csv('/content/testset/english/Wikipedia_Test.tsv',sep='\t')

# Combine the DataFrames into a single training set and a single test set
train = pd.concat([news_train, wikinews_train,wikipedia_train], ignore_index=True)
train.dropna(axis=1, how='all')
validation = pd.concat([news_dev, wikinews_dev,wikipedia_dev], ignore_index=True)
validation.dropna(axis=1, how='all', inplace=True)
test=pd.concat([news_test,wikinews_test,wikipedia_test],ignore_index=True)
test.dropna(axis=1, how='all', inplace=True)

In [ ]:
feature_labels = ['id', 'sentence', "start", "end", "target", 
           "nat", "non_nat", "nat_marked", "non_nat_marked"]
target_label = 'target'

# Rename the columns using the labels
train.columns = feature_labels + [target_label]

# Separate the features (input variables) from the labels (output variable)
X = train[feature_labels]
y = train[target_label]

ValueError: ignored

In [ ]:
train


,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops are believed to be in rich in oil and gas and the overlapping claims have long been feared as Asia's next flashpoint for armed conflict.",4,10,barren,10.1,10.2,6,2,1,...,16,Guatemalan,0,1.1,0.05,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",8,Normally,0.1
0,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",4.0,18,barren islands,10,10.0,0.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",20.0,25,reefs,10,10.0,1.0,2.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",11.0,18,islands,10,10.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",30.0,35,coral,10,10.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",30.0,44,coral outcrops,10,10.0,2.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27291,NaN,NaN,NaN,10,NaN,10,NaN,NaN,NaN,0,...,NaN,NaN,123.0,0.0,0.0,3ZUE82NE0A2B8701X4995O8OCRS8FL,An actor ( sometimes actress for female ; see ...,128.0,works,0.0
27292,NaN,NaN,NaN,10,NaN,10,NaN,NaN,NaN,0,...,NaN,NaN,132.0,0.0,0.0,3ZUE82NE0A2B8701X4995O8OCRS8FL,An actor ( sometimes actress for female ; see ...,136.0,film,0.0
27293,NaN,NaN,NaN,10,NaN,10,NaN,NaN,NaN,0,...,NaN,NaN,139.0,0.0,0.0,3ZUE82NE0A2B8701X4995O8OCRS8FL,An actor ( sometimes actress for female ; see ...,149.0,television,0.0
27294,NaN,NaN,NaN,10,NaN,10,NaN,NaN,NaN,0,...,NaN,NaN,152.0,0.0,0.0,3ZUE82NE0A2B8701X4995O8OCRS8FL,An actor ( sometimes actress for female ; see ...,159.0,theater,0.0


In [ ]:
validation

In [ ]:
test